In [ ]:
import pandas as pd
import numpy as np
mentees = pd.read_csv('~/Downloads/Cohort 3 Data - Mentees.csv')
mentors = pd.read_csv('~/Downloads/Cohort 3 Data - Mentors.csv')
#Clean DF
mentees_flitered = mentees.filter(items=["Id",
                 "What's your email?",
                 'What role are you looking for help in?',
                 "Which best describes the position you're in?",
                 'How many years of professional experience do you have in your current role?',
                 'What is the main goal you want help with from your mentor?',
                 'How many years of experience do you have as a founder?',
                 'What industry are you looking for help with?',
                 'What company stage do you want your mentor to be in?',
                 'What timezone are you in?',
                 'Have you been matched before?',
                 "Tell us why you want to be a part of Lenny's Mentorship Program!",
                 'What is the main goal you want help with from your mentor?'
                ])

mentors_flitered = mentors.filter(items=["Id",
                 "What's your email?",
                 'Which functional role do you want your mentee to be in?',
                 "Which best describes the position you're in?",
                 'How many years of professional experience do you have in the role you want to be a mentor for?',
                 'Which goals can you help your mentee with?',
                 'How many years of experience do you have as a founder?',
                 'What industry can you help your mentee with?',
                 'What company stage do you want your mentee to be in?',
                 'What timezone are you in?',
                 'Have you been matched before?',
                 'What goals can you help your mentee with?'
                ])
#print(mentors.columns.values)
#print(mentors_flitered.columns.values)
#display(mentees_flitered)
#display(mentors_flitered)


In [ ]:
#Input comma seperated list of value
#Output list of values with whitespace stipped off
def clean_multiselect(x):
    if isinstance(x, str):
        return list(map(str.strip,x.split(',')))
    else:
        return []


In [ ]:
#Input Dataframe and multi-select field to Binarize
from sklearn.preprocessing import MultiLabelBinarizer
def MultiLableBinarize_df(input_frame, collumn_name):
    nested_list = list(map(clean_multiselect,input_frame[collumn_name].to_list()))
    mlb = MultiLabelBinarizer()
    mlb_df = pd.DataFrame(mlb.fit_transform(nested_list), columns=mlb.classes_)
    bigger = pd.concat([input_frame,mlb_df],axis=1)
    return bigger


In [ ]:
class multiSelect:
    def __init__(self, data = ['empty']):
        if isinstance(data, str):
            self.data = clean_multiselect(data)
        else:
            self.data = data
    def __repr__(self):
        return repr(self.data)

In [ ]:
class distanceEstimator:
    def __init__(x):
        1==1
    def estimateDistance(self, row):
        #print(mentors.columns.values)
        mentee = row['What industry are you looking for help with?'].data
        mentor = row['What industry can you help your mentee with?'].data

        distance_score = 1000
        if isinstance(mentee,list) and isinstance(mentor,list):
            for industry in mentor:
                if industry in mentee:
                    distance_score = distance_score - 50
        return distance_score

In [ ]:
mentees_flitered['What industry are you looking for help with?'] = mentees_flitered['What industry are you looking for help with?'].apply(multiSelect)
mentors_flitered['What industry can you help your mentee with?'] = mentors_flitered['What industry can you help your mentee with?'].apply(multiSelect)

combined = mentors_flitered.join(mentees_flitered,how='cross',lsuffix='-mentor',rsuffix='-mentee')

In [ ]:
dE = distanceEstimator()
combined['distance_score'] = combined.apply(dE.estimateDistance, axis = 'columns')
combined = combined.sort_values(by=['distance_score'])
#display(combined)

In [ ]:
matched_mentors = {}
matched_mentees = {}
matched_list = []
for index, row in combined.iterrows():
    if row['Id-mentor'] not in matched_mentors:
        matched_mentors[row['Id-mentor']] = 0
    if row['Id-mentee'] not in matched_mentees:
        matched_mentees[row['Id-mentee']] = 0
    if matched_mentors[row['Id-mentor']] >=1:
        continue
    if matched_mentees[row['Id-mentee']] >=1:
        continue
    matched_mentors[row['Id-mentor']] = matched_mentors[row['Id-mentor']] + 1
    matched_mentees[row['Id-mentee']] = matched_mentees[row['Id-mentee']] + 1
    matched_list.append({'Id-mentor':row['Id-mentor'],'Id-mentee':row['Id-mentee'], 'distance_score':row['distance_score']})
print(matched_list)
#print(matched_mentors)
#print(matched_mentees)

[{'Id-mentor': 319, 'Id-mentee': 224.0, 'distance_score': 900}, {'Id-mentor': 316, 'Id-mentee': 24.0, 'distance_score': 900}, {'Id-mentor': 321, 'Id-mentee': 49.0, 'distance_score': 900}, {'Id-mentor': 328, 'Id-mentee': 136.0, 'distance_score': 900}, {'Id-mentor': 314, 'Id-mentee': 73.0, 'distance_score': 900}, {'Id-mentor': 97, 'Id-mentee': 139.0, 'distance_score': 900}, {'Id-mentor': 87, 'Id-mentee': 87.0, 'distance_score': 900}, {'Id-mentor': 69, 'Id-mentee': 69.0, 'distance_score': 900}, {'Id-mentor': 231, 'Id-mentee': 58.0, 'distance_score': 900}, {'Id-mentor': 212, 'Id-mentee': 117.0, 'distance_score': 900}, {'Id-mentor': 216, 'Id-mentee': 118.0, 'distance_score': 900}, {'Id-mentor': 301, 'Id-mentee': 26.0, 'distance_score': 900}, {'Id-mentor': 200, 'Id-mentee': 187.0, 'distance_score': 900}, {'Id-mentor': 21, 'Id-mentee': 171.0, 'distance_score': 900}, {'Id-mentor': 302, 'Id-mentee': 201.0, 'distance_score': 900}, {'Id-mentor': 2, 'Id-mentee': 72.0, 'distance_score': 900}, {'Id-